# Carole's playground

In [1]:
import requests
import json
import pandas as pd

In [2]:
apikey = 'BuiXohTN1i'
baseUrl = 'http://data.ons.gov.uk/ons/api/data'
apikKeyParam = 'apikey={}'.format(apikey);

url = 'http://data.ons.gov.uk/ons/api/data/.json?apikey=' + apikey

## Available contexts

In [3]:
url = '{}/contexts.json?{}'.format(baseUrl, apikKeyParam)
print(url)
resp = requests.get(url)
contexts =json.loads(resp.text)
for context in contexts['ons']['contextList']['statisticalContext']:
    print(context)

http://data.ons.gov.uk/ons/api/data/contexts.json?apikey=BuiXohTN1i
{'contextId': 1, 'contextName': 'Census'}
{'contextId': 2, 'contextName': 'Socio-Economic'}
{'contextId': 3, 'contextName': 'Economic'}
{'contextId': 4, 'contextName': 'Social'}


## Concepts
The context must be set to Census

In [4]:
context = 'Census'
url = '{}/concepts.json?{}&context={}'.format(baseUrl, apikKeyParam, context)
url

'http://data.ons.gov.uk/ons/api/data/concepts.json?apikey=BuiXohTN1i&context=Census'

In [5]:
r = requests.get(url)
if(r.status_code != 200):
    print('Error returned for {}: {}'.format(r.url, r.status_code))
    print(r.text)
else:
    raw = json.loads(r.text)
    concepts = [concept for concept in raw['ons']['conceptList']['concept']]
    temp = {'name':[], 'id':[]}
    temp['id'] = [concept['id'] for concept in concepts]
    temp['name'] = [[name['$'] for name in concept['names']['name'] if name['@xml.lang'] == 'en'][0] for concept in concepts]
    conceptsdf = pd.DataFrame(temp)
    conceptsdf['id'] = conceptsdf['id'].astype(int)

In [6]:
conceptsdf

,id,name
0,76,Family type
1,75,Marital and civil partnership staus
2,30,Sex
3,29,Second address
4,28,Residence type
5,27,Population density
6,26,People sleeeping rough
7,25,Living arrangements
8,24,Household deprivation dimensions
9,23,Household composition
